In [2]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [1]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model='gpt-3.5-turbo')

In [4]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content='my favourite color is blue')
]

In [5]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content='Blue is a calming and peaceful color. It is often associated with stability and trustworthiness. What do you like most about the color blue?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 12, 'total_tokens': 41, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-938664b5-afc7-4933-ae45-cb544b525084-0', usage_metadata={'input_tokens': 12, 'output_tokens': 29, 'total_tokens': 41})

In [7]:
chatbot.invoke('what is my favourite color?')

AIMessage(content="I'm sorry, but I do not have the information about your favorite color. Only you would know what your favorite color is.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 13, 'total_tokens': 39, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-21f17f04-bcca-48fd-97e8-1a527bb7e4f8-0', usage_metadata={'input_tokens': 13, 'output_tokens': 26, 'total_tokens': 39})

## Lets add memory to the chatbot

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbot_memory =  {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in chatbot_memory:
        chatbot_memory[session_id] = ChatMessageHistory()
    return chatbot_memory[session_id]

chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot,
    get_session_history
)

In [15]:
session1 = { 'configurable': {'session_id': '001'}}

responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='My favourite color is red')],
    config=session1
)
responseFromChatbot.content

"That's a great choice! Red is often associated with passion, energy, and love. What do you like most about the color red?"

In [19]:
chatbot_memory

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favourite color is red', additional_kwargs={}, response_metadata={}), AIMessage(content="That's a great choice! Red is often associated with passion, energy, and love. What do you like most about the color red?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 12, 'total_tokens': 40, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e681c706-c8f9-45a4-a2f5-aa01cdcdef16-0', usage_metadata={'input_tokens': 12, 'output_tokens': 28, 'total_tokens': 40}), HumanMessage(content='what is my favorite color?', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on your previous statement, it seems like your favorite color is red! Is that correct?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'compl

In [17]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='what is my favorite color?')],
    config=session1
)

In [18]:
responseFromChatbot.content

'Based on your previous statement, it seems like your favorite color is red! Is that correct?'

In [20]:
session2 = { 'configurable': {'session_id': '002'}}

responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='what is my favourite color?')],
    config=session2
)
responseFromChatbot.content

"I'm sorry, as an AI assistant, I do not have access to personal information about you, such as your favorite color."

In [25]:
chatbot_memory['001'].messages

[HumanMessage(content='My favourite color is red', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's a great choice! Red is often associated with passion, energy, and love. What do you like most about the color red?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 12, 'total_tokens': 40, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e681c706-c8f9-45a4-a2f5-aa01cdcdef16-0', usage_metadata={'input_tokens': 12, 'output_tokens': 28, 'total_tokens': 40}),
 HumanMessage(content='what is my favorite color?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Based on your previous statement, it seems like your favorite color is red! Is that correct?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 54, '

* **The memory of a chatbot is included in the context window of the LLM so, if left unmanaged, can potentialy overflow it**
* **We are now going to learn how to limit the size of the memory of a chatbot**

In [35]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate,  MessagesPlaceholder

def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [(
        "system",
        "You are a helpful assistant. Answer all questions to the best of your capacities"
    ),
    MessagesPlaceholder(variable_name='messages')]
)

limited_memory_chain = (
    RunnablePassthrough.assign(messages=lambda x : limited_memory_of_messages(x['messages']))
    | prompt
    | chatbot
)

In [36]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limited_memory_chain,
    get_session_history,
    input_messages_key='messages'
)

In [29]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='I like Vespas')],
    config=session1
)
responseFromChatbot.content

'Vespas are stylish and iconic scooters! They come in a variety of colors, including red. Do you have a favorite color for your Vespa?'

In [30]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='My favourite ones are red colored')],
    config=session1
)
responseFromChatbot.content

"Red Vespas are definitely eye-catching and classic! It's great that your favorite color matches your favorite mode of transportation. Do you have a red Vespa or are you planning to get one in the future?"

In [31]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='I like cinema')],
    config=session1
)
responseFromChatbot.content

"That's wonderful! Cinema is a great form of entertainment and art. Do you have a favorite movie genre or film director?"

In [37]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        'messages': [HumanMessage(content='what is my favourite color?')]
    },
    config=session1
)
responseFromChatbot.content

"I'm sorry, but as an AI assistant, I don't have access to personal information about users unless it has been shared with me in the course of our conversation. If you've mentioned your favorite color previously, I can try to help you remember it."